In [2]:
import torch_explain as te
from torch_explain.logic import test_explanation
from torch_explain.logic.nn import explain_class
import torch
from torch.nn.functional import one_hot


# XOR problem with additional features
x0 = torch.zeros((4, 100))
x = torch.tensor([
[0, 0, 0],
[0, 1, 0],
[1, 0, 0],
[1, 1, 0],
], dtype=torch.float)
x = torch.cat([x, x0], dim=1)
y = torch.tensor([0, 1, 1, 0], dtype= torch.long)

# network architecture
layers = [
    te.nn.EntropyLogicLayer(x.shape[1], n_classes=2),
    torch.nn.LeakyReLU(),
    te.nn.LinearIndependent(10, 10, n_classes=2),
    torch.nn.LeakyReLU(),
    te.nn.LinearIndependent(10, 1, n_classes=2, top=True)
]
model = torch.nn.Sequential(*layers)

# train loop
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
loss_form = torch.nn.CrossEntropyLoss()
model.train()
for epoch in range(1001):
    optimizer.zero_grad()
    y_pred = model(x)
    loss = loss_form(y_pred, y) + \
    0.00001 * te.nn.functional.entropy_logic_loss(model)
    loss.backward()
    optimizer.step()

# logic explanations
y1h = one_hot(y)

_, class_explanations, _ = explain_class(model, x, y1h, x, y1h)


AttributeError: module 'torch_explain.nn' has no attribute 'EntropyLogicLayer'